In [1]:
import os
import csv
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim 
from torchvision import transforms, datasets
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from datetime import datetime
import torchvision.models as models
import random
import numpy as np
import gc
import wandb

In [2]:
# 画像のサイズ
IMG_SIZE = 224

# データのディレクトリのパス
FLOWER_DIR = '../input/flowers'

batch_size = 128


In [3]:
# GPUが利用可能かどうかを確認し、利用可能ならGPUを使用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


Using device: cuda


# データセットの作成

In [4]:
transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)), # 画像サイズをリサイズ,
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# データセットを作成
dataset = datasets.ImageFolder(FLOWER_DIR, transform=transforms)


In [5]:
# 学習データに使用する割合
n_train_ratio = 0.9

# 割合から個数を出す
n_train = int(len(dataset) * n_train_ratio)
n_val   = int(len(dataset) - n_train)


In [6]:
# 学習データと検証データに分割
train, val = random_split(dataset, [n_train, n_val])


In [7]:
# Data Loader
train_loader = DataLoader(train, batch_size, shuffle=True, drop_last=True, num_workers=2)
val_loader = DataLoader(val, batch_size, num_workers=2)


In [8]:
# ResNet-18モデルの定義
class CustomResNet18(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet18, self).__init__()
        self.resnet = models.resnet18(weights=None)  # 事前学習済みのResNet-18モデルをロード
        # 最終の全結合層を入れ替えて、出力クラスの数に合わせる
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# ResNet-34モデルの定義
class CustomResNet34(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet34, self).__init__()
        self.resnet = models.resnet34(weights=None)  # 事前学習済みのResNet-34モデルをロード
        # 最終の全結合層を入れ替えて、出力クラスの数に合わせる
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# ResNet-50モデルの定義
class CustomResNet50(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet50, self).__init__()
        self.resnet = models.resnet50(weights=None)  # 事前学習済みのResNet-34モデルをロード
        # 最終の全結合層を入れ替えて、出力クラスの数に合わせる
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet(x)


In [9]:
# 2. モデルの構築
model1 = CustomResNet18(num_classes=5)
model2 = CustomResNet34(num_classes=5)
model3 = CustomResNet50(num_classes=5)

In [10]:
def train_and_evaluate_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        # 訓練フェーズ
        model.train()
        train_loss, train_correct = 0, 0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}, Training...'):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()

        # 検証フェーズ
        model.eval()
        val_loss, val_correct = 0, 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs}, Validation...'):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()


        scheduler.step(val_loss)
        # 現在の学習率を取得
        current_lr = optimizer.param_groups[0]['lr']

        # Calculate losses and accuracies
        train_loss_epoch = train_loss / len(train_loader)
        val_loss_epoch = val_loss / len(val_loader)
        train_acc_epoch = train_correct / len(train_loader.dataset)
        val_acc_epoch = val_correct / len(val_loader.dataset)

        train_losses.append(train_loss_epoch)
        val_losses.append(val_loss_epoch)
        train_accuracies.append(train_acc_epoch)
        val_accuracies.append(val_acc_epoch)

        # Log metrics to wandb
        wandb.log({"epoch": epoch, "train_loss": train_loss_epoch, "val_loss": val_loss_epoch, 
                   "train_accuracy": train_acc_epoch, "val_accuracy": val_acc_epoch, 
                   "learning_rate": current_lr})

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]}, Val Loss: {val_losses[-1]}, Train Acc: {train_accuracies[-1]}, Val Acc: {val_accuracies[-1]}')

    return train_losses, val_losses, train_accuracies, val_accuracies


In [11]:
def clear_gpu():
    torch.cuda.empty_cache()
    gc.collect()

In [12]:
models = [model1, model2, model3]
model_names = ["ResNet18", "ResNet34", "ResNet50"]

In [13]:
for model, model_name in zip(models, model_names):
    wandb.init(project="comparing with resnet models", name = model_name, reinit = True)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
    num_epochs = 50

    model.to(device)
    train_losses, val_losses, train_accuracies, val_accuracies = train_and_evaluate_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

    clear_gpu()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chibachann. Use `wandb login --relogin` to force relogin


Epoch 1/50, Validation...: 100%|██████████| 4/4 [00:00<00:00,  5.17it/s]


Epoch 1/50, Train Loss: 1.2948746800422668, Val Loss: 1.956826090812683, Train Acc: 0.4882882882882883, Val Acc: 0.36574074074074076


Epoch 2/50, Validation...: 100%|██████████| 4/4 [00:00<00:00,  5.75it/s]


Epoch 2/50, Train Loss: 0.9912045220534007, Val Loss: 1.1006075590848923, Train Acc: 0.6041184041184041, Val Acc: 0.5648148148148148


...

Epoch 48/50, Train Loss: 0.09415319884816806, Val Loss: 0.47617966681718826, Train Acc: 0.957014157014157, Val Acc: 0.8703703703703703


Epoch 49/50, Validation...: 100%|██████████| 4/4 [00:00<00:00,  4.57it/s]


Epoch 49/50, Train Loss: 0.08631589909394582, Val Loss: 0.5226010158658028, Train Acc: 0.9619047619047619, Val Acc: 0.8541666666666666


Epoch 50/50, Validation...: 100%|██████████| 4/4 [00:00<00:00,  4.46it/s]


Epoch 50/50, Train Loss: 0.08713895181814829, Val Loss: 0.5129712074995041, Train Acc: 0.9606177606177606, Val Acc: 0.8611111111111112


In [17]:
wandb.init(project="comparing with resnet models", name = "ResNet50", reinit = True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
num_epochs = 50

model3.to(device)
train_losses, val_losses, train_accuracies, val_accuracies = train_and_evaluate_model(model3, train_loader, val_loader, criterion, optimizer, num_epochs)

clear_gpu()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▅█
learning_rate,▁▁▁
train_accuracy,▁▅█
train_loss,█▃▁
val_accuracy,▁▃█
val_loss,▂█▁
epoch,2
learning_rate,0.001
train_accuracy,0.87671
train_loss,0.32667
val_accuracy,0.80324


Epoch 1/50, Validation...: 100%|██████████| 4/4 [00:01<00:00,  3.48it/s]


Epoch 1/50, Train Loss: 1.4768320202827454, Val Loss: 1.3926225304603577, Train Acc: 0.4404118404118404, Val Acc: 0.5


Epoch 2/50, Validation...: 100%|██████████| 4/4 [00:01<00:00,  3.67it/s]


...

Epoch 48/50, Train Loss: 0.09334098224838575, Val Loss: 0.5682697035372257, Train Acc: 0.9575289575289575, Val Acc: 0.8194444444444444


Epoch 49/50, Validation...: 100%|██████████| 4/4 [00:01<00:00,  3.68it/s]


Epoch 49/50, Train Loss: 0.08603931019703547, Val Loss: 0.611153319478035, Train Acc: 0.9619047619047619, Val Acc: 0.8240740740740741


Epoch 50/50, Validation...: 100%|██████████| 4/4 [00:01<00:00,  3.52it/s]


Epoch 50/50, Train Loss: 0.08614380694925786, Val Loss: 0.6263664811849594, Train Acc: 0.963963963963964, Val Acc: 0.8101851851851852


In [14]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
learning_rate,████████████████████████▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train_accuracy,▁▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇███████████████
train_loss,█▆▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▄▅▆▅▅▄▅▆▆▆▆▆▅▇▇▇▇▇▆▇▇████████████████
val_loss,█▃▃▄▄▃▄▅▆▃▂▂▂▂▂▄▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,49
learning_rate,0.0
train_accuracy,0.96062
train_loss,0.08714
val_accuracy,0.86111
